In [1]:
pip install langchain_community

In [4]:
import getpass
import os
api_key = getpass.getpass("OpenAI API キーを入力してください: ")
os.environ["OPENAI_API_KEY"] = api_key

api_key1 = getpass.getpass("SERPAPI API キーを入力してください: ")
os.environ["SERPAPI_API_KEY"] = api_key1

OpenAI API キーを入力してください:  ········
SERPAPI API キーを入力してください:  ········


In [6]:
pip install langchainhub

In [8]:
from langchain import hub

prompt = hub.pull("hwchase17/react")

print(prompt.template)

C:\anaconda\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


# おみくじツール定義（再掲）

In [11]:
import random
from datetime import datetime

def get_fortune(date_string):
    # 日付文字列を解析
    try:
        date = datetime.strptime(date_string, "%m月%d日")
    except ValueError:
        return "無効な日付形式です。'X月X日'の形式で入力してくださ\
い。"

    # 運勢のリスト
    fortunes = [
        "大吉", "中吉", "小吉", "吉", "末吉", "凶", "大凶"
    ]

    # 運勢の重み付け（大吉と大凶の確率を低くする）
    weights = [1, 3, 3, 4, 3, 2, 1]

    # 日付に基づいてシードを設定（同じ日付なら同じ運勢を返す）
    random.seed(date.month * 100 + date.day)

    # 運勢をランダムに選択
    fortune = random.choices(fortunes, weights=weights)[0]

    return f"{date_string}の運勢は【{fortune}】です。"

# ツール作成

from langchain.tools import BaseTool

class Get_fortune(BaseTool):
    name: str = 'Get_fortune'
    description: str = (
        "特定の日付の運勢を占う。インプットは  'date_string'です。\
'date_string' は、占いを行う日付で、mm月dd日 という形式です。「1月1日」\
のように入力し、「'1月1日'」のように余計な文字列を付けてはいけません。"
    )

    def _run(self, date_string) -> str:
        return get_fortune(date_string)


    async def _arun(self, query: str) -> str:
        raise NotImplementedError("does not support async")

# 日付ツール定義（再掲）

In [14]:
from datetime import timedelta
from zoneinfo import ZoneInfo


def get_date(date):
    date_now = datetime.now(ZoneInfo("Asia/Tokyo"))
    if ("今日" in date):
        date_delta = 0
    elif ("明日" in date):
        date_delta = 1
    elif ("明後日" in date):
        date_delta = 2
    else:
        return "サポートしていません"
    return (date_now + timedelta(days=date_delta)).strftime\
('%m月%d日')

class Get_date(BaseTool):
    name: str = 'Get_date'
    description: str = (
        "今日の日付を取得する。インプットは 'date'です。'date' は、日\
付を取得する対象の日で、'今日', '明日', '明後日' という3種類の文字列\
から指定します。「今日」のように入力し、「'今日'」のように余計な文字列を付\
けてはいけません。"
    )

    def _run(self, date) -> str:
        return get_date(date)

    async def _arun(self, query: str) -> str:
        raise NotImplementedError("does not support async")

# エージェントの作成

In [17]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.agents import AgentExecutor, create_react_agent

model = ChatOpenAI(model="gpt-4o-mini")

tools = [Get_date(), Get_fortune()]

#1 エージェントの作成
agent = create_react_agent(model, tools, prompt)

#2 エージェントの実行準備
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


response = agent_executor.invoke({"input": [HumanMessage\
(content="今日の運勢を教えてください。")]})

print(response)



> Entering new AgentExecutor chain...
I need to obtain today's date in order to provide a fortune for today.  
Action: Get_date  
Action Input: 今日  05月07日I have obtained today's date, which is May 7th. Now I need to get the fortune for this specific date.  
Action: Get_fortune  
Action Input: 5月07日  5月07日の運勢は【吉】です。I now know the final answer.  
Final Answer: 今日の運勢は【吉】です。

> Finished chain.
{'input': [HumanMessage(content='今日の運勢を教えてください。', additional_kwargs={}, response_metadata={})], 'output': '今日の運勢は【吉】です。'}


# 検索できるエージェント作成

In [20]:
from langchain.agents import load_tools

model = ChatOpenAI(model="gpt-4o-mini")
tools = load_tools(["serpapi"], llm=model)
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


response = agent_executor.invoke({"input": [HumanMessage\
 (content="株式会社Elithの住所を教えてください。最新の公式情報として公\
 開されているものを教えてください。")]})

print(response)



> Entering new AgentExecutor chain...
株式会社Elithの住所について最新の公式情報を調べる必要があります。公式のウェブサイトや最新の情報源を検索します。  
Action: Search  
Action Input: "株式会社Elith 住所 公式サイト"  ['株式会社エリスの公式HPです。 Elithは、最先端のAI技術をビジネス実装し価値を生み出す松尾研発スタートアップです.', '\u200b会社概要 · \u200bElith OFFICE · 東京都文京区本郷3-30-10本郷 K&Kビル1F · 東京メトロ丸ノ内線 / 本郷三丁目駅 徒歩約2分 都営大江戸線 / 本郷三丁目駅 徒歩約3分 東京メトロ ...', '会社概要株式会社Elithは、東大松尾研発スタートアップ(R)として、最先端のAI ... 住所. 日本113-0033東京都文京区本郷3-30-10本郷 K&Kビル1F. この出展社を見ている ...', '連絡先. https://www.elith.co.jp/ contact@elith.co.jp. 住所. 日本113-0033東京都文京区本郷3-30-10本郷 K&Kビル1F. この出展社を見ている人は、こちらの出展社も見てい ...', '法人番号: 6011001151296; 法人名: 株式会社Elith; 法人名ふりがな: えりす; 法人名英語: 本店所在地: 東京都文京区本郷3丁目30番10号本郷K＆Kビル1F ...', '株式会社アイスマイリーのプレスリリース（2024年7月17日 09時00分）AI博覧会、第6弾スピーカーを発表！Elith 井上氏、メロン 本田氏が講演！', '株式会社Elithの概要. 住所, 〒 〒113-0033 東京都文京区本郷3-30-10本郷 K&Kビル1F. 分野, 映像認識・画像解析（視覚的）, 音声・音による入力（聴覚的）. 業界, 医療 ...', '東大・松尾研発スタートアップの株式会社Elith(エリス)の公式アカウントです。 AI開発・研究を行っています。気軽にリプください、たくさん交流しましょう！', '- 会社名：株式会社Elith · - 代表者：若林怜帆人（代表取締役）、井上顧基（取締役CEO兼CTO） · - 所在